In [27]:
import numpy as np
np.set_printoptions(linewidth = 150, precision = 4, suppress = True)

In [2]:
data = np.loadtxt('../NC-Data.csv', delimiter=',', dtype=str)
data = data[1:].astype(float)
data

array([[ 1.    , -0.0461,  0.2312, ...,  0.2704,  0.4664,  0.3672],
       [-0.0461,  1.    , -0.0671, ..., -0.0515, -0.0944, -0.0349],
       [ 0.2312, -0.0671,  1.    , ...,  0.147 ,  0.2608,  0.6313],
       ...,
       [ 0.2704, -0.0515,  0.147 , ...,  1.    ,  0.2066,  0.1538],
       [ 0.4664, -0.0944,  0.2608, ...,  0.2066,  1.    ,  0.3486],
       [ 0.3672, -0.0349,  0.6313, ...,  0.1538,  0.3486,  1.    ]])

In [3]:
nodes = np.loadtxt('../NC-K7-Trace-Nodes.csv', delimiter=',', dtype=str)
nodes = nodes[1:].astype(float)
bounds = np.loadtxt('../NC-K7-Trace-Bounds.csv', delimiter=',', dtype=str)
bounds = bounds[1:].astype(float)

In [4]:
np.argmax((nodes == 1).sum(axis=0) == 3)

42

In [5]:
bounds[:, 42]

array([4.1355, 5.8707, 7.    ])

In [6]:
selected = np.where(nodes[:, 42] == 1)[0]
selected

array([ 3, 83, 85])

In [1]:
import scipy.special
def ncr(n,k):
    return int(scipy.special.binom(n,k))
ncr(10,3)

120

In [18]:
def combination(n, p, x):
    result = [None for i in range(p)]
    r = 0
    k = 0
    for i in range(p-1):
        result[i] = result[i-1] if i>0 else 0
        result[i] += 1
        r = ncr(n-result[i], p-(i+1))
        k += r
        while k < x:
            result[i] += 1
            r = ncr(n-result[i], p-(i+1))
            k += r
        k -= r
    result[p-1] = result[p-2] + x - k
    return result

In [24]:
import tensorflow as tf

In [63]:
def pascal(n, max_k=None):
    max_k = n if max_k is None else max_k
    result = np.zeros([n,n], np.int64)
    result[:, 0] = 1
    for i in range(1, n):
        for j in range(1, min(i+1, max_k)):
            result[i, j] = result[i-1, j-1] + result[i-1, j]
    return result

In [64]:
ncr = tf.constant(pascal(101, 10))

In [66]:
ncr.numpy().max()

1902231808400

In [82]:
@tf.function(jit_compile=True)
def combination(n, p, x):
    result = tf.TensorArray(ncr.dtype, size=p)
    r = tf.constant(0, ncr.dtype)
    k = tf.constant(0, ncr.dtype)
    next_value = tf.constant(0, ncr.dtype)
    for i in tf.range(p-1):
        next_value += 1
        index = tf.cast(i, tf.int64)
        r = ncr[n-next_value, p-(index+1)]
        k += r
        while k < x:
            next_value += 1
            r = ncr[n-next_value, p-(index+1)]
            k += r
        k -= r
        result = result.write(i, next_value)
    result = result.write(p-1, next_value + x - k)
    return result.stack()

In [77]:
@tf.function(jit_compile=True)
def combinations_batch(n, k, start, limit):
    return tf.map_fn(
        lambda i: combination(n, k, i),
        tf.range(start, limit),
        dtype=tf.int64)

In [50]:
combination(tf.constant(10), 3, tf.constant(15))

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([ 1,  3, 10], dtype=int32)>

In [85]:
scipy.special.binom(10, 8)

45.0

In [86]:
combinations_batch(tf.constant(10,tf.int64), 8, tf.constant(0,tf.int64), tf.constant(45,tf.int64))

<tf.Tensor: shape=(45, 8), dtype=int64, numpy=
array([[ 1,  2,  3,  4,  5,  6,  7,  7],
       [ 1,  2,  3,  4,  5,  6,  7,  8],
       [ 1,  2,  3,  4,  5,  6,  7,  9],
       [ 1,  2,  3,  4,  5,  6,  7, 10],
       [ 1,  2,  3,  4,  5,  6,  8,  9],
       [ 1,  2,  3,  4,  5,  6,  8, 10],
       [ 1,  2,  3,  4,  5,  6,  9, 10],
       [ 1,  2,  3,  4,  5,  7,  8,  9],
       [ 1,  2,  3,  4,  5,  7,  8, 10],
       [ 1,  2,  3,  4,  5,  7,  9, 10],
       [ 1,  2,  3,  4,  5,  8,  9, 10],
       [ 1,  2,  3,  4,  6,  7,  8,  9],
       [ 1,  2,  3,  4,  6,  7,  8, 10],
       [ 1,  2,  3,  4,  6,  7,  9, 10],
       [ 1,  2,  3,  4,  6,  8,  9, 10],
       [ 1,  2,  3,  4,  7,  8,  9, 10],
       [ 1,  2,  3,  5,  6,  7,  8,  9],
       [ 1,  2,  3,  5,  6,  7,  8, 10],
       [ 1,  2,  3,  5,  6,  7,  9, 10],
       [ 1,  2,  3,  5,  6,  8,  9, 10],
       [ 1,  2,  3,  5,  7,  8,  9, 10],
       [ 1,  2,  3,  6,  7,  8,  9, 10],
       [ 1,  2,  4,  5,  6,  7,  8,  9],
       [ 1

In [69]:
scipy.special.binom(20, 8)

125970.0

In [84]:
%timeit combinations_batch(tf.constant(20,tf.int64), 8, tf.constant(0,tf.int64), tf.constant(125970,tf.int64))

14.5 ms ± 42 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
